In [37]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import time
import os
import requests 

import pandas as pd
from pandas import DataFrame, read_csv
import scipy as sc
import numpy as np

import re

from sklearn.preprocessing import scale

import statsmodels.formula.api as sm

import matplotlib.pyplot as plt 
plt.style.use('ggplot')
%matplotlib inline  
plt.rcParams['figure.figsize'] = (10, 6) 

In [38]:
ncaaStats = pd.read_csv('Data/CSV Files/Database/FromCollegeToNBA.csv')
careerTotPlayer = pd.read_csv('Data/CSV Files/Database/careerAVGPlayerWithKMeans.csv')

In [ ]:
careerTotPlayer.columns = ['ID', 'Player', 'Years', 'From', 'To', 'G', 'MP', 'FG', 'FGA', 'FG%',
       '3P', '3PA', '3P%', 'FTM', 'FTA', 'FT%', 'TOV', 'PF', 'ORB', 'DRB',
       'REB', 'AST', 'STL', 'BLK', 'PTS']
careerTotPlayer

In [3]:
careerTotPlayer = careerTotPlayer[['Player', 'Group']]
newTable = pd.merge(ncaaStats, careerTotPlayer, on='Player', how='inner')
del newTable['Team']
newTable

,ID,Player,Years,From,To,GP,MIN,FGM,FGA,FG%,...,DRB,REB,AST,STL,BLK,PTS,2P%,eFG%,Draft,Group
0,456,Troy Bell,1.0,2003.0,2003.0,31,38.645,7.226,16.387,44.095930,...,3.032,4.581,3.710,2.258,0.226,25.194,48.367425,54.527979,16,0
1,452,Carmelo Anthony,1.0,2003.0,2003.0,35,36.400,7.914,17.486,45.259064,...,6.857,9.743,2.200,1.486,0.857,22.229,49.548772,49.834153,3,3
2,444,Mike Sweetney,1.0,2003.0,2003.0,34,32.000,7.765,14.206,54.660003,...,6.706,10.765,1.941,1.382,3.206,22.824,55.000708,54.660003,9,1
3,410,Andre Emmett,2.0,2003.0,2004.0,68,37.015,8.162,15.956,51.153171,...,4.456,6.647,1.838,1.559,0.382,21.235,52.697659,52.397217,35,0
4,417,Luis Flores,2.0,2003.0,2004.0,61,37.492,7.525,16.770,44.871795,...,3.410,4.803,2.246,1.934,0.246,24.311,48.137435,50.933214,55,0
5,450,Dwyane Wade,1.0,2003.0,2003.0,33,32.061,7.606,15.182,50.098801,...,4.515,6.333,4.394,2.152,1.303,21.515,51.859340,51.495192,5,3
6,292,Nick Collison,1.0,2003.0,2003.0,38,32.421,7.395,13.342,55.426473,...,6.711,10.000,2.211,1.158,1.947,18.474,57.146330,56.708140,12,1
7,57,Chris Kaman,1.0,2003.0,2003.0,31,34.032,7.871,12.645,62.245947,...,8.419,12.032,1.258,0.613,3.161,22.419,62.245947,62.245947,6,2
8,474,Willie Green,1.0,2003.0,2003.0,30,34.033,8.133,16.600,48.993976,...,3.867,4.933,2.467,1.333,0.433,22.600,51.751294,52.707831,41,1
9,21671,Willie Green,3.0,2012.0,2014.0,91,25.077,3.703,7.088,52.243228,...,3.154,5.055,1.374,0.484,1.330,9.758,55.334170,53.484763,41,1


In [4]:
newTable['Draft'][newTable['Draft'] == 'Undrafted'] = 61

C:\Users\Moshe Gotam\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [5]:
newTable

,ID,Player,Years,From,To,GP,MIN,FGM,FGA,FG%,...,DRB,REB,AST,STL,BLK,PTS,2P%,eFG%,Draft,Group
0,456,Troy Bell,1.0,2003.0,2003.0,31,38.645,7.226,16.387,44.095930,...,3.032,4.581,3.710,2.258,0.226,25.194,48.367425,54.527979,16,0
1,452,Carmelo Anthony,1.0,2003.0,2003.0,35,36.400,7.914,17.486,45.259064,...,6.857,9.743,2.200,1.486,0.857,22.229,49.548772,49.834153,3,3
2,444,Mike Sweetney,1.0,2003.0,2003.0,34,32.000,7.765,14.206,54.660003,...,6.706,10.765,1.941,1.382,3.206,22.824,55.000708,54.660003,9,1
3,410,Andre Emmett,2.0,2003.0,2004.0,68,37.015,8.162,15.956,51.153171,...,4.456,6.647,1.838,1.559,0.382,21.235,52.697659,52.397217,35,0
4,417,Luis Flores,2.0,2003.0,2004.0,61,37.492,7.525,16.770,44.871795,...,3.410,4.803,2.246,1.934,0.246,24.311,48.137435,50.933214,55,0
5,450,Dwyane Wade,1.0,2003.0,2003.0,33,32.061,7.606,15.182,50.098801,...,4.515,6.333,4.394,2.152,1.303,21.515,51.859340,51.495192,5,3
6,292,Nick Collison,1.0,2003.0,2003.0,38,32.421,7.395,13.342,55.426473,...,6.711,10.000,2.211,1.158,1.947,18.474,57.146330,56.708140,12,1
7,57,Chris Kaman,1.0,2003.0,2003.0,31,34.032,7.871,12.645,62.245947,...,8.419,12.032,1.258,0.613,3.161,22.419,62.245947,62.245947,6,2
8,474,Willie Green,1.0,2003.0,2003.0,30,34.033,8.133,16.600,48.993976,...,3.867,4.933,2.467,1.333,0.433,22.600,51.751294,52.707831,41,1
9,21671,Willie Green,3.0,2012.0,2014.0,91,25.077,3.703,7.088,52.243228,...,3.154,5.055,1.374,0.484,1.330,9.758,55.334170,53.484763,41,1


In [6]:
newTable['Draft'] = newTable['Draft'].astype(float)
newTable.dtypes

ID          int64
Player     object
Years     float64
From      float64
To        float64
GP          int64
MIN       float64
FGM       float64
FGA       float64
FG%       float64
3PM       float64
3PA       float64
3P%       float64
FTM       float64
FTA       float64
FT%       float64
TOV       float64
PF        float64
ORB       float64
DRB       float64
REB       float64
AST       float64
STL       float64
BLK       float64
PTS       float64
2P%       float64
eFG%      float64
Draft     float64
Group       int64
dtype: object

In [7]:
newTable.columns

Index(['ID', 'Player', 'Years', 'From', 'To', 'GP', 'MIN', 'FGM', 'FGA', 'FG%',
       '3PM', '3PA', '3P%', 'FTM', 'FTA', 'FT%', 'TOV', 'PF', 'ORB', 'DRB',
       'REB', 'AST', 'STL', 'BLK', 'PTS', '2P%', 'eFG%', 'Draft', 'Group'],
      dtype='object')

In [8]:
toScale = ['Years', 'From', 'To', 'GP', 'MIN', 'FGM', 'FGA', 'FG%',
       '3PM', '3PA', '3P%', 'FTM', 'FTA', 'FT%', 'TOV', 'PF', 'ORB', 'DRB',
       'REB', 'AST', 'STL', 'BLK', 'PTS', '2P%', 'eFG%', 'Draft']

In [9]:
for x in range(0, len(toScale)):
    newTable[toScale[x]] = scale(newTable[toScale[x]])

C:\Users\Moshe Gotam\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)


In [42]:
newTable

,ID,Player,Years,From,To,GP,MIN,FGM,FGA,FG%,...,DRB,REB,AST,STL,BLK,PTS,2P%,eFG%,Draft,Group
0,456,Troy Bell,-1.312087,-1.407556,-1.738266,-1.342324,2.303620,2.147377,2.521053,-0.571353,...,-0.544921,-0.378021,1.003209,2.491315,-0.675131,3.321261,-0.509097,0.421034,-1.134035,0
1,452,Carmelo Anthony,-1.312087,-1.407556,-1.738266,-1.230214,1.768603,2.694562,2.929150,-0.381306,...,2.155738,1.986093,-0.016510,0.887173,0.136640,2.521283,-0.299475,-0.575298,-1.756222,3
2,444,Mike Sweetney,-1.312087,-1.407556,-1.738266,-1.258241,0.720018,2.576058,1.711171,1.154730,...,2.049123,2.454153,-0.191416,0.671071,3.158590,2.681818,0.667936,0.449058,-1.469059,1
3,410,Andre Emmett,-0.371054,-1.407556,-1.518181,-0.305304,1.915166,2.891803,2.361007,0.581742,...,0.460501,0.568174,-0.260973,1.038860,-0.474439,2.253095,0.259274,-0.031251,-0.224684,0
4,417,Luis Flores,-0.371054,-1.407556,-1.518181,-0.501497,2.028843,2.385180,2.663274,-0.444583,...,-0.278033,-0.276349,0.014554,1.818074,-0.649401,3.083021,-0.549908,-0.342006,0.732527,0
5,450,Dwyane Wade,-1.312087,-1.407556,-1.738266,-1.286269,0.734555,2.449601,2.073594,0.409467,...,0.502158,0.424367,1.465122,2.271057,0.710412,2.328641,0.110520,-0.222719,-1.660501,3
6,292,Nick Collison,-1.312087,-1.407556,-1.738266,-1.146131,0.820348,2.281788,1.390337,1.279965,...,2.052654,2.103795,-0.009082,0.205621,1.538907,1.508158,1.048662,0.883804,-1.325477,1
7,57,Chris Kaman,-1.312087,-1.407556,-1.738266,-1.342324,1.204273,2.660363,1.131516,2.394211,...,3.258595,3.034419,-0.652653,-0.926836,3.100698,2.572546,1.953556,2.059282,-1.612640,2
8,474,Willie Green,-1.312087,-1.407556,-1.738266,-1.370352,1.204512,2.868738,2.600147,0.228948,...,0.044634,-0.216811,0.163798,0.569254,-0.408829,2.621382,0.091348,0.034681,0.062479,1
9,21671,Willie Green,0.569978,0.571446,0.682659,0.339329,-0.929836,-0.654558,-0.931993,0.759849,...,-0.458783,-0.160937,-0.574317,-1.194886,0.745147,-0.843480,0.727106,0.199596,0.062479,1


In [43]:
d = newTable.describe()

In [51]:
d[d.index == 'mean']

,ID,Years,From,To,GP,MIN,FGM,FGA,FG%,3PM,...,DRB,REB,AST,STL,BLK,PTS,2P%,eFG%,Draft,Group
mean,10181.583261,-1.452965e-15,1.516877e-14,4.655848e-15,-2.185961e-16,1.306979e-15,6.845258e-16,-3.885302e-16,1.863144e-16,-3.357491e-17,...,-4.218656e-16,-4.105622e-16,4.235898e-16,-2.086338e-16,-4.665044e-17,-1.810459e-16,-2.459206e-16,-2.486267e-16,-6.523398e-17,0.78516


In [11]:
X = newTable.iloc[:, 5:27].values
y = newTable.iloc[:, 28].values

# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [12]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(927, 22) (927,)
(232, 22) (232,)


In [13]:
from sklearn import preprocessing
X_scaled = preprocessing.scale(X_train)
# X_test_scaled = preprocessing.scale(X_test)

In [14]:
from numpy import *

where_are_NaNs = isnan(X_scaled)
X_scaled[where_are_NaNs] = 0
where_are_NaNs = isnan(X_test)
X_test[where_are_NaNs] = 0
# where_are_NaNs = isnan(X_scaled)
# X_scaled[where_are_NaNs] = X_scaled.mean()

In [53]:
from sklearn import linear_model
lm = linear_model.LinearRegression()
model = lm.fit(X_train, y_train)
predictions = lm.predict(X_scaled)

In [54]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier

estimator = DecisionTreeClassifier(max_leaf_nodes=5, random_state=0)
estimator.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=5, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=0, splitter='best')

In [55]:
# Using those arrays, we can parse the tree structure:

n_nodes = estimator.tree_.node_count
children_left = estimator.tree_.children_left
children_right = estimator.tree_.children_right
feature = estimator.tree_.feature
threshold = estimator.tree_.threshold

In [56]:
# The tree structure can be traversed to compute various properties such
# as the depth of each node and whether or not it is a leaf.
node_depth = np.zeros(shape=n_nodes, dtype=np.int64)
is_leaves = np.zeros(shape=n_nodes, dtype=bool)
stack = [(0, -1)]  # seed is the root node id and its parent depth
while len(stack) > 0:
    node_id, parent_depth = stack.pop()
    node_depth[node_id] = parent_depth + 1

    # If we have a test node
    if (children_left[node_id] != children_right[node_id]):
        stack.append((children_left[node_id], parent_depth + 1))
        stack.append((children_right[node_id], parent_depth + 1))
    else:
        is_leaves[node_id] = True

print("The binary tree structure has %s nodes and has "
      "the following tree structure:"
      % n_nodes)
for i in range(n_nodes):
    if is_leaves[i]:
        print("%snode=%s leaf node." % (node_depth[i] * "\t", i))
    else:
        print("%snode=%s test node: go to node %s if X[:, %s] <= %s else to "
              "node %s."
              % (node_depth[i] * "\t",
                 i,
                 children_left[i],
                 feature[i],
                 threshold[i],
                 children_right[i],
                 ))
print()

The binary tree structure has 9 nodes and has the following tree structure:
node=0 test node: go to node 1 if X[:, 0] <= -0.7677592039108276 else to node 2.
	node=1 test node: go to node 3 if X[:, 20] <= -1.135667085647583 else to node 4.
	node=2 leaf node.
		node=3 leaf node.
		node=4 test node: go to node 5 if X[:, 6] <= 1.5199283957481384 else to node 6.
			node=5 test node: go to node 7 if X[:, 10] <= -2.460934638977051 else to node 8.
			node=6 leaf node.
				node=7 leaf node.
				node=8 leaf node.



In [57]:
from sklearn import metrics
from sklearn import tree


decisionTree = tree.DecisionTreeClassifier()

In [59]:
# fit the tree with the traing data
decisionTree = decisionTree.fit(X_train, y_train)

In [60]:
y_pred_train = decisionTree.predict(X_train)
# measure accuracy
print('Accuracy on training data = ', metrics.accuracy_score(y_true = y_train, y_pred = y_pred_train))

# predict with the test data
y_pred = decisionTree.predict(X_test)
# measure accuracy
print('Accuracy on test data = ', metrics.accuracy_score(y_true = y_test, y_pred = y_pred))

Accuracy on training data =  0.9859762675296656
Accuracy on test data =  0.34051724137931033


In [61]:
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.tree import DecisionTreeClassifier
prediction = decisionTree.predict(X_test)
print(confusion_matrix(y_test,prediction))
print('\n')
print(classification_report(y_test,prediction))

[[42 44 17  1]
 [44 32 15  2]
 [ 8 13  5  2]
 [ 2  2  3  0]]


              precision    recall  f1-score   support

           0       0.44      0.40      0.42       104
           1       0.35      0.34      0.35        93
           2       0.12      0.18      0.15        28
           3       0.00      0.00      0.00         7

   micro avg       0.34      0.34      0.34       232
   macro avg       0.23      0.23      0.23       232
weighted avg       0.35      0.34      0.35       232



In [62]:
from IPython.display import Image, display  
from scipy import misc

import pydotplus
import collections
# Visualize data
dot_data = tree.export_graphviz(decisionTree,
                                out_file=None,
                                filled=True,
                                rounded=True)

graph = pydotplus.graph_from_dot_data(dot_data)

colors = ('turquoise', 'orange')
edges = collections.defaultdict(list)

for edge in graph.get_edge_list():
    edges[edge.get_source()].append(int(edge.get_destination()))

for edge in edges:
    edges[edge].sort()    
    for i in range(2):
        dest = graph.get_node(str(edges[edge][i]))[0]
        dest.set_fillcolor(colors[i])

graph.write_png('tree.png')
display(Image('tree.png'))

## NB

In [63]:
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB

In [64]:
where_are_NaNs = isnan(X_train)
X_train[where_are_NaNs] = 0

In [65]:
# Instantiate the classifier
gnb = GaussianNB()

# Train classifier
gnb.fit(X_train,y_train)

y_pred = gnb.predict(X_test)
y_pred_train = gnb.predict(X_train)

# Print results
print('Accuracy on Train data= ', metrics.accuracy_score(y_true = y_train, y_pred = y_pred_train))
print('Accuracy on test data= ', metrics.accuracy_score(y_true = y_test, y_pred = y_pred))

Accuracy on Train data=  0.42934196332254587
Accuracy on test data=  0.40086206896551724
